In [1]:
from langchain_ollama import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore

from langchain_core.documents import Document
from langchain_ollama import OllamaLLM

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
import json
import pandas as pd
import ollama
import ragas

In [2]:
gdpr_articles_json_filename = 'Output/gdpr_articles_24_43_qa_output.json'

with open(gdpr_articles_json_filename) as json_file:
    article_json_data = json.load(json_file)
print(len(article_json_data))

122


In [3]:
gdpr_recitals_json_filename = 'Output/gdpr_chapter4_recital_qa_correct_output.json'

with open(gdpr_recitals_json_filename) as json_file:
    recital_json_data = json.load(json_file)
print(len(recital_json_data))

46


In [4]:
embeddings = OllamaEmbeddings(
model='all-minilm',
)


In [5]:
article_vector_store = InMemoryVectorStore(embeddings)



In [6]:

documents= []
for index, article in enumerate(article_json_data):
   
    content = article["InputText"]
    identifier =  str(index+1)+":article:" + article['Article']
    
    document_1 = Document(id=identifier, page_content=content,metadata={"id": identifier})
    print(document_1.metadata)
    
    documents.append(document_1)
    
article_vector_store.add_documents(documents)


print(len(article_vector_store.store.items()))    

{'id': '1:article:24'}
{'id': '2:article:24'}
{'id': '3:article:25'}
{'id': '4:article:25'}
{'id': '5:article:25'}
{'id': '6:article:26'}
{'id': '7:article:26'}
{'id': '8:article:27'}
{'id': '9:article:27'}
{'id': '10:article:28'}
{'id': '11:article:28'}
{'id': '12:article:28'}
{'id': '13:article:28'}
{'id': '14:article:28'}
{'id': '15:article:28'}
{'id': '16:article:28'}
{'id': '17:article:28'}
{'id': '18:article:28'}
{'id': '19:article:28'}
{'id': '20:article:28'}
{'id': '21:article:28'}
{'id': '22:article:28'}
{'id': '23:article:28'}
{'id': '24:article:28'}
{'id': '25:article:28'}
{'id': '26:article:29'}
{'id': '27:article:30'}
{'id': '28:article:30'}
{'id': '29:article:30'}
{'id': '30:article:30'}
{'id': '31:article:30'}
{'id': '32:article:30'}
{'id': '33:article:30'}
{'id': '34:article:30'}
{'id': '35:article:30'}
{'id': '36:article:30'}
{'id': '37:article:30'}
{'id': '38:article:30'}
{'id': '39:article:30'}
{'id': '40:article:30'}
{'id': '41:article:31'}
{'id': '42:article:32'}
{

In [7]:
for index, (id, doc) in enumerate(article_vector_store.store.items()):
    print(id)
    print(doc['id'])
    
    print(doc['metadata'])
    break

1:article:24
1:article:24
{'id': '1:article:24'}


In [8]:
from langchain_chroma import Chroma

In [9]:

chroma_art_vector_store = Chroma.from_documents(documents, embeddings)

In [7]:
#sample_recital_question = "What is the exemption from record-keeping requirements for?"
sample_recital_question = "what types of harm can result from personal data processing?"

input = sample_recital_question
# Use the vectorstore as a retriever

retriever = article_vector_store.as_retriever(
    search_kwargs={"k": 3}
)

'''
retriever = chroma_art_vector_store.as_retriever(
    #search_type="mmr",
    search_kwargs={ "k": 4}
)
'''

# use chroma as vector store 

# Retrieve the most similar text
retrieved_documents = retriever.invoke(input)
#print(retrieved_documents)
print(len(retrieved_documents))
# Show the retrieved document's content

for doc in retrieved_documents:
    print(doc.id)
    print(doc.page_content)
    print(f'{doc.metadata}')
    

3
58:article:35
Where a type of processing in particular using new technologies, and taking into account the nature, scope, context and purposes of the processing, is likely to result in a high risk to the rights and freedoms of natural persons, the controller shall, prior to the processing, carry out an assessment of the impact of the envisaged processing operations on the protection of personal data.
{'id': '58:article:35'}
55:article:34
The communication to the data subject referred to in paragraph 1 of this Article shall describe in clear and plain language the nature of the personal data breach and contain at least the information and measures referred to in points (b) (c) and (d) of Article 33 (3)
{'id': '55:article:34'}
61:article:35
A data protection impact assessment referred to in paragraph 1 shall in particular be required in the case of: (a) a systematic and extensive evaluation of personal aspects relating to natural persons which is based on automated processing, includin

In [10]:
use_concise=True

system_prompt = (
    "Use only the given context to answer the question. "
    "Context: {context}"
)

if use_concise:
 
    system_prompt = (
    "Use only the given context provide a concise answer to the following question"
    "Context: {context}"
    )   
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
llm = OllamaLLM(model="llama3:8b")


question_answer_chain = create_stuff_documents_chain(llm, prompt)
# not able to use this wrapped retreiver in a chain 
#chain = create_retrieval_chain(wrapped_retriever, question_answer_chain)
chain = create_retrieval_chain(retriever, question_answer_chain)

print("sample query:")
print(sample_recital_question)

result = chain.invoke({"input": sample_recital_question})

context_docs = result['context']
for context_doc in context_docs:
    print(context_doc.id)
    print(context_doc.page_content)
    print(context_doc.metadata)
    
print(result['answer'])

sample query:
what types of harm can result from personal data processing?
58:article:35
Where a type of processing in particular using new technologies, and taking into account the nature, scope, context and purposes of the processing, is likely to result in a high risk to the rights and freedoms of natural persons, the controller shall, prior to the processing, carry out an assessment of the impact of the envisaged processing operations on the protection of personal data.
{'id': '58:article:35'}
55:article:34
The communication to the data subject referred to in paragraph 1 of this Article shall describe in clear and plain language the nature of the personal data breach and contain at least the information and measures referred to in points (b) (c) and (d) of Article 33 (3)
{'id': '55:article:34'}
61:article:35
A data protection impact assessment referred to in paragraph 1 shall in particular be required in the case of: (a) a systematic and extensive evaluation of personal aspects rel

In [12]:

# different prompt

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

llm = OllamaLLM(model="llama3:8b")


question_answer_chain = create_stuff_documents_chain(llm, prompt)
# not able to use this wrapped retreiver in a chain 
#chain = create_retrieval_chain(wrapped_retriever, question_answer_chain)
chain = create_retrieval_chain(retriever, question_answer_chain)

print("sample query:")
print(sample_recital_question)

result = chain.invoke({"input": sample_recital_question})

context_docs = result['context']
for context_doc in context_docs:
    print(context_doc.id)
    print(context_doc.page_content)
    print(context_doc.metadata)
    
print(result['answer'])

sample query:
what types of harm can result from personal data processing?
58:article:35
Where a type of processing in particular using new technologies, and taking into account the nature, scope, context and purposes of the processing, is likely to result in a high risk to the rights and freedoms of natural persons, the controller shall, prior to the processing, carry out an assessment of the impact of the envisaged processing operations on the protection of personal data.
{'id': '58:article:35'}
55:article:34
The communication to the data subject referred to in paragraph 1 of this Article shall describe in clear and plain language the nature of the personal data breach and contain at least the information and measures referred to in points (b) (c) and (d) of Article 33 (3)
{'id': '55:article:34'}
61:article:35
A data protection impact assessment referred to in paragraph 1 shall in particular be required in the case of: (a) a systematic and extensive evaluation of personal aspects rel

In [13]:
print(result)

{'input': 'what types of harm can result from personal data processing?', 'context': [Document(id='58:article:35', metadata={'id': '58:article:35'}, page_content='Where a type of processing in particular using new technologies, and taking into account the nature, scope, context and purposes of the processing, is likely to result in a high risk to the rights and freedoms of natural persons, the controller shall, prior to the processing, carry out an assessment of the impact of the envisaged processing operations on the protection of personal data.'), Document(id='55:article:34', metadata={'id': '55:article:34'}, page_content='The communication to the data subject referred to in paragraph 1 of this Article shall describe in clear and plain language the nature of the personal data breach and contain at least the information and measures referred to in points (b) (c) and (d) of Article 33 (3)'), Document(id='61:article:35', metadata={'id': '61:article:35'}, page_content='A data protection 

In [17]:

kg_data = pd.read_excel("Output/gdpr_articles_24_43_kg_output_corrected.xlsx")

kg_article_vector_store = InMemoryVectorStore(embeddings)

kg_list = [] 
for kg_row in kg_data.iterrows():
    kg_row = kg_row[1]
    kg_list.append(kg_row)
    
kg_documents= []
# store each document in a vector embedding database
for index, (article, kg) in enumerate(zip(article_json_data,kg_list)):
       
    identifier =  str(index+1)+":article:" + article['Article']

    content = kg
    doc_dict = content.to_dict()
    #print(doc_dict)
    content=json.dumps(doc_dict)
    document_1 = Document(id=identifier, page_content=content)
    
    kg_documents.append(document_1)

print(len(kg_documents))
kg_article_vector_store.add_documents(kg_documents)


print(len(kg_article_vector_store.store.items()))    

122
122


In [19]:
kg_retriever = kg_article_vector_store.as_retriever(
    search_kwargs={"k": 3}
)


In [20]:

for index in range(len(recital_json_data)):
    print(index)
       
    recital_query = recital_json_data[index]['question']
    recital_query= "What kind of measures should a controller establish to demonstrate compliance with this Regulation?"
    query = {}
    #query['InputText'] = recital_row['question']
    query['InputText'] = recital_query
    query['Type'] =''  
    query['Action'] =''
    
    query['Condition'] =''                                                     
    query['Modality'] =''                        
    query['Actor'] =''                                                         
    query['Reason'] =''        
    query['Artifact'] =''      
    query['Violation'] =''                                                
    query['Exception'] =''                                                
    query['Presence'] =''                                                 
    query['Time'] =''                                                     
    query['Sanction'] =''                                                 
    query['Situation'] =''
    query_data = json.dumps(query)
    recital_query= query_data
    print(query_data)
    retrieved_documents = kg_retriever.invoke(recital_query)
    print(len(retrieved_documents))

    for doc in retrieved_documents:
        print(doc.id)
        print(doc.page_content)
    break

0
{"InputText": "What kind of measures should a controller establish to demonstrate compliance with this Regulation?", "Type": "", "Action": "", "Condition": "", "Modality": "", "Actor": "", "Reason": "", "Artifact": "", "Violation": "", "Exception": "", "Presence": "", "Time": "", "Sanction": "", "Situation": ""}
3
67:article:35
{"InputText": "Where appropriate, the controller shall seek the views of data subjects or their representatives on the intended processing, without prejudice to the protection of commercial or public interests or the security of processing operations. ", "Type": "Obligation", "Action": "seek the views of data subjects or their representatives on the intended processing", "Condition": "where appropriate", "Modality": "shall", "Actor": "controller", "Reason": " ", "Artifact": NaN, "Violation": NaN, "Exception": NaN, "Presence": NaN, "Time": NaN, "Sanction": NaN, "Situation": NaN}
2:article:24
{"InputText": "Where proportionate in relation to processing activitie

In [ ]:
from ragas import EvaluationDataset


dataset = []

for index,recital in enumerate(recital_json_data):
    if index== 1:
        break
    print(index)
    recital_question = recital['question']
    recital_answer= recital['answer']
    print("recital query:")
    print(recital_question)
    print("recital answer:")
    print(recital_answer)

    result = chain.invoke({"input": recital_question})

    print("LLM response")

    context_docs = result['context']
    '''
    for context_doc in context_docs:
        print(context_doc.id)
        
        print(context_doc.page_content)
        #print(context_doc.metadata)
    '''

    print("response")
    print(result['answer'])

    dataset.append(
        {
            "user_input": recital_question,
            "retrieved_contexts": [rdoc.page_content for rdoc in context_docs],
            "response": result['answer'],
            "reference": recital_answer,
        }
    )

    

evaluation_dataset = EvaluationDataset.from_list(dataset)
    

In [19]:
from ragas import evaluate,RunConfig
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

tiny_llm = OllamaLLM(model="tinyllama")
#tiny_llm = OllamaLLM(model="tinyllama", request_timeout=120.0) 

#evaluator_llm = LangchainLLMWrapper(tiny_llm)

# Create a RunConfig with a specified timeout
run_config = RunConfig(timeout=300, max_retries=20)  # Set timeout to 120 seconds

#run_config =RunConfig(timeout=600, max_retries=20, max_wait=50,log_tenacity=False),


result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
   run_config=run_config
)

print(result)


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

Exception raised in Job[0]: TimeoutError()
Exception raised in Job[1]: TimeoutError()
Exception raised in Job[2]: TimeoutError()


{'context_recall': nan, 'faithfulness': nan, 'factual_correctness(mode=f1)': nan}


In [ ]:
from ragas import evaluate,RunConfig
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
# tried using openai but too many calls
from langchain_openai import OpenAI
eval_llm = OpenAI(
    api_key="",
    model="gpt-4o-mini"
)


evaluator_llm = LangchainLLMWrapper(eval_llm)
# Create a RunConfig with a specified timeout
run_config = RunConfig(timeout=300)  # Set timeout to 120 seconds

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
    run_config=run_config
)

print(result)

alternative prompt

In [52]:

recital_query= "What kind of measures should a controller establish to demonstrate compliance with this Regulation?"
  
# Define a question
question = recital_query

# Retrieve relevant documents
docs = retriever.invoke(question)

# Combine the documents into a single string
docs_text = "\n".join(d.page_content for d in docs)

print(docs_text)

promptText = f"""
        You are an assistant for question-answering tasks. 
        Use the following pieces of retrieved context to answer the question. 
        {docs_text}  provide a concise answer to the given query: {question}
         """
        #generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
          model="llama3:8b",
          prompt=promptText
        )
        #print(promptText)

print("response from LLM:")

print(output['response'])
        

Where proportionate in relation to processing activities, the measures referred to in paragraph 1 shall include the implementation of appropriate data protection policies by the controller.  Adherence to approved codes of conduct as referred to in Article 40 or approved certification mechanisms as referred to in Article 42 may be used as an element by which to demonstrate compliance with the obligations of the controller.
Taking into account the nature, scope, context and purposes of processing as well as the risks of varying likelihood and severity for the rights and freedoms of natural persons, the controller shall implement appropriate technical and organisational measures to ensure and to be able to demonstrate that processing is performed in accordance with this Regulation.  Those measures shall be reviewed and updated where necessary.
the processor makes available to the controller all information necessary to demonstrate compliance with the obligations laid down in this Article 

Experiment using contextual data only answer the recital questions

In [ ]:
from deepeval.dataset import EvaluationDataset
from deepeval.test_case import LLMTestCase, LLMTestCaseParams




dataset = []

for index,recital in enumerate(recital_json_data):
    if index> 50:
        break
    print(index)
    recital_question = recital['question']
    recital_answer= recital['answer']
    print("recital query:")
    print(recital_question)
    print("recital answer:")
    print(recital_answer)

    result = chain.invoke({"input": recital_question})

    print("LLM response")

    context_docs = result['context']

    '''
    for context_doc in context_docs:
        print(context_doc.id)
        
        print(context_doc.page_content)
        print(context_doc.metadata)
    '''

    #print("response")
    print(result['answer'])

    retrieved_contexts = [rdoc.page_content for rdoc in context_docs]

    test_case = LLMTestCase(input=recital_question, expected_output =recital_answer,actual_output=result['answer'], retrieval_context= retrieved_contexts)
    
    dataset.append(test_case)
    


dataset = EvaluationDataset(test_cases=dataset)

    

In [23]:
def get_closest_match_kg(recital_question):
    
    fields = ['InputText', 'Action','Reason', 'all']
    article_kgs = []
    article_kg_set = set()
    print(recital_question)
    for field in fields:
        print(field)
        query = {}
        #query['InputText'] = recital_row['question']
        query['InputText'] = ''
        query['Type'] =''  
        query['Action'] =''
        
        query['Condition'] =''                                                     
        query['Modality'] =''                        
        query['Actor'] =''                                                         
        query['Reason'] =''        
        query['Artifact'] =''      
        query['Violation'] =''                                                
        query['Exception'] =''                                                
        query['Presence'] =''                                                 
        query['Time'] =''                                                     
        query['Sanction'] =''                                                 
        query['Situation'] =''

        if field == all:
            query[field[0]] = recital_question
            query[field[1]] = recital_question
            query[field[2]] = recital_question

        else:
            query[field] = recital_question
        
        query_data = json.dumps(query)
        recital_query= query_data
        
        retrieved_documents = kg_retriever.invoke(recital_query)

        print(len(retrieved_documents))

        for doc in retrieved_documents:
            print(doc.id)
            article_kg=doc.page_content
        
            article_kg_set.add(article_kg)
       
            article_kg=json.loads(article_kg)
            article_kgs.append(article_kg)
        
        
            print("article kg")
            print(article_kg)
    print(len(article_kg_set))       
    return article_kg_set

In [25]:
sample_recital_query= "What is the exemption from record-keeping requirements for?"
kg_set=get_closest_match_kg(sample_recital_query)

What is the exemption from record-keeping requirements for?
InputText
3
38:article:30
article kg
{'InputText': 'The records referred to in paragraphs 1 and 2 shall be in writing, including in electronic form.', 'Type': 'Obligation', 'Action': 'be in writing, including in electronic form', 'Condition': nan, 'Modality': 'shall', 'Actor': ' ', 'Reason': nan, 'Artifact': '[paragraphs 1 and 2]', 'Violation': nan, 'Exception': nan, 'Presence': nan, 'Time': nan, 'Sanction': nan, 'Situation': nan}
28:article:30
article kg
{'InputText': 'The record  of processing activities shall contain the purposes of the processing; ', 'Type': 'Obligation', 'Action': 'contain the purposes of the processing', 'Condition': nan, 'Modality': 'shall', 'Actor': ' ', 'Reason': nan, 'Artifact': '[the record]', 'Violation': nan, 'Exception': nan, 'Presence': nan, 'Time': nan, 'Sanction': nan, 'Situation': nan}
5:article:25
article kg
{'InputText': 'An approved certification mechanism pursuant to Article 42 may be use

In [12]:
from deepeval.metrics import (
  ContextualRelevancyMetric,
  ContextualRecallMetric,
  ContextualPrecisionMetric,
  AnswerRelevancyMetric,
  FaithfulnessMetric
)

from deepeval.metrics import GEval

correctness_metric = GEval(
        name="Correctness",
        criteria="Determine if the 'actual output' is correct based on the 'expected output'.",
        evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
        threshold=0.5
    )

from deepeval import evaluate

contextual_precision = ContextualPrecisionMetric()
contextual_recall = ContextualRecallMetric()
contextual_relevancy = ContextualRelevancyMetric()
answer_relevancy = AnswerRelevancyMetric(threshold=0.8)
faithfulness = FaithfulnessMetric()

#evaluate(dataset, metrics=[contextual_precision, contextual_recall, contextual_relevancy, answer_relevancy, faithfulness])

#evaluate(dataset, metrics=[ answer_relevancy])
#evaluation_output = evaluate(dataset, metrics=[correctness_metric,answer_relevancy])
evaluation_output = evaluate(dataset, metrics=[correctness_metric])

#dataset.evaluate([answer_relevancy])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using llama3:8b (Ollama), strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ❌ Correctness [GEval] (score: 0.2, threshold: 0.5, strict: False, evaluation model: llama3:8b (Ollama), reason: The actual output partially aligns with the expected output by mentioning the importance of ensuring compliance with the Regulation. However, it lacks specific details about providing legal certainty and transparency for economic operators, natural persons, and supervisory authorities, which are key aspects mentioned in the expected output., error: None)

For test case:

  - input: What is the primary objective of introducing a Regulation to ensure consistent protection for natural persons throughout the Union?
  - actual output: The primary objective of introducing this Regulation is to ensure that personal data is processed in accordance with the Regulation, and that controllers and processors can demonstrate compliance with it. This includes implementing appropriate technical and organisational measures to ensure that processing is performed in accor

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

In [13]:
test_results = evaluation_output.test_results

In [14]:
print(len(test_results))

46


In [15]:
sorted_test_results = sorted(test_results, key=lambda x: int(x.name.replace("test_case_","")))

In [16]:
print("Answer correctness")
number_success=0
for test in sorted_test_results:
    print(test.name)
    print(test.input)
    print(test.actual_output)
    print(test.expected_output)
    print("%s" %test.metrics_data[0].success)
    if (test.metrics_data[0].success==True):
        
        number_success+=1
    print(test.metrics_data[0].score)
    print(test.metrics_data[0].reason)
    print(" ")
    

print("number Correctness successful:%d" %number_success)



Answer correctness
test_case_0
What is the primary objective of introducing a Regulation to ensure consistent protection for natural persons throughout the Union?
The primary objective of introducing this Regulation is to ensure that personal data is processed in accordance with the Regulation, and that controllers and processors can demonstrate compliance with it. This includes implementing appropriate technical and organisational measures to ensure that processing is performed in accordance with the Regulation, reviewing and updating these measures as necessary.
to provide legal certainty and transparency for economic operators, including micro, small and medium-sized enterprises and to provide natural persons in all Member states with the same legal of legally enforceable rights and obligations and responsibilities for controllers and processors, to ensure consistent monitoring of the processing of personal data, and equivalent sanctions in all Member States as well as effective coo

In [52]:
print("Answer relevancy")
number_success=0
for test in sorted_test_results:
    print(test.name)
    print(test.input)
    print(test.actual_output)
    print(test.expected_output)
    print("%s" %test.metrics_data[1].success)
    if (test.metrics_data[1].success==True):
        
        number_success+=1
    print(test.metrics_data[1].score)
    print(test.metrics_data[1].reason)
    print(" ")
    

print("number Answer Relevancy successful:%d" %number_success)


Answer relevancy
test_case_0
What is the primary objective of introducing a Regulation to ensure consistent protection for natural persons throughout the Union?
Based on the provided context, the primary objective of introducing this Regulation is to ensure "processing is performed in accordance with" it, which means that the controller must implement appropriate technical and organisational measures to guarantee the processing is done in line with the Regulation's requirements.
to provide legal certainty and transparency for economic operators, including micro, small and medium-sized enterprises and to provide natural persons in all Member states with the same legal of legally enforceable rights and obligations and responsibilities for controllers and processors, to ensure consistent monitoring of the processing of personal data, and equivalent sanctions in all Member States as well as effective cooperation between the supervisory authorities of different Member States.
True
1.0
The s

In [45]:
print(test)

TestResult(name='test_case_45', success=False, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=False, score=0.2, reason='The actual output partially aligns with the expected output by mentioning certification mechanisms and data protection seals, but it does not directly address enhancing transparency. The actual output also provides more context about the goal of demonstrating compliance, which is not present in the expected output.', strict_mode=False, evaluation_model='llama3:8b (Ollama)', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\nDetermine if the \'actual output\' is correct based on the \'expected output\'. \n \nEvaluation Steps:\n[\n    "Evaluate the \'actual output\' against the \'expected output\'.",\n    "Determine if the \'actual output\' matches the \'expected output\' in terms of content, format, and structure.",\n    "If the \'actual output\' is correct, verify that it meets all requirements outlined in the \'expected output\'.",